# Data Preparation for ML

Before training any ML model, you will have to invest some time in data preparation. There are usually two steps:

- Data Cleaning is very general and usually needs to be done before training. This step should be applied iteratively during EDA.
- Data Transformations are usually very ML task/algorithm specific. The goal is to convert data into a suitable form for the task/algorithm.

> ✏️ The example is inspired by {cite}`packtpublishing`, {cite}`scikit_preprocessing` and {cite}`scikit_imputation`.

Let's start by importing packages:

In [ ]:
# TODO
import pandas as pd

## Data Cleaning

This section will show you some techniques for dealing with noisy datasets. In general, there are two approaches:

- Throwing noisy instances away
- Applying some techniques to data to reduce the noise

Usually, throwing data away is not the best option because acquiring data (and additional samples) is very expensive, so we try to salvage as much data as possible. Of course, it should only be done to some extent because, in many cases, some samples are beyond recovery).

### Handling Duplications

In many cases, data handed to us might contain duplicated instances (for example, a single measurement is duplicated several times). Having such data might introduce a lot of unwanted biases.

In [ ]:
# TODO

### Handling Duplications

In many cases, data handed to us might contain duplicated instances (for example, a single measurement is duplicated several times). Having such data might introduce a lot of unwanted biases.

In [ ]:
# TODO

### Handling missing or incorrect data

It is very often that we receive a noisy dataset with some incorrect or missing measurements. There are many ways how to approach the problem.

#### Dropping instances or attributes

As discussed earlier, removing information should be done cautiously; let's review some examples:

In [ ]:
# TODO

#### Replacing values

This is the easiest option if you have any domain knowledge of how wrong or missing values should be treated.

In [ ]:
# TODO

#### Filling in missing data

Let's start with some basics techniques:

In [ ]:
# TODO

There are also more sophisticated options:

In [ ]:
# TODO

### Data Transformations

Let's review some techniques that should help you to transform data into a form that is required by the ML task/algorithm.

#### Discretization

Discretization (known as quantization or binning) allows the partitioning of continuous features into discrete values. Specific datasets may benefit from discretization.

In [ ]:
# TODO

#### Encoding Variables

Often features are not given as continuous values but as categorical. ML algorithms do not always support such representation. In that case, we might need to apply different encodings:

In [ ]:
# TODO

#### Standardization, or mean removal, variance scaling, and non-linear transformations

Many ML algorithms might have some requirements for input distributions of features. It might also be beneficial to transform a feature distribution as it can significantly improve convergence speed or lead to better performance. Let's review some techniques:

## Exercises

#### Try to add some new features as a combination of the available ones to a dataset. Do a simple EDA to analyze patterns.

Often it’s helpful to add complexity to a model by considering nonlinear features of the input data. Review implementation of [polynomial features](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html#sklearn.preprocessing.PolynomialFeatures) and apply it to the Titanic dataset.

In [ ]:
# load the dataset
titanic_url = 'https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv'
titanic_data = pd.read_csv(titanic_url)

# TODO: your answer here

## Resources

```{bibliography}
:filter: docname in docnames
```